In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("anthony-wss/uniform-with-aug")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

In [2]:
df = pd.json_normalize([summary_list[i] | config_list[i] for i in range(len(summary_list))], sep='_')
df.columns

Index(['ure', 'scel', 'test_acc', 'training_loss', '_timestamp', '_step',
       '_runtime', 'train_acc', 'valid_acc', 'lr', 'algo', 'seed', 'model',
       'data_aug', 'dataset_name', 'data_cleaning_rate',
       'best_epoch-scel_valid_acc', 'best_epoch-scel_training_loss',
       'best_epoch-scel_ure', 'best_epoch-scel_scel', 'best_epoch-scel_epoch',
       'best_epoch-scel_test_acc', 'best_epoch-scel_train_acc',
       'best_epoch-ure_scel', 'best_epoch-ure_epoch',
       'best_epoch-ure_test_acc', 'best_epoch-ure_train_acc',
       'best_epoch-ure_valid_acc', 'best_epoch-ure_training_loss',
       'best_epoch-ure_ure', 'best_epoch-valid_acc_epoch',
       'best_epoch-valid_acc_test_acc', 'best_epoch-valid_acc_train_acc',
       'best_epoch-valid_acc_valid_acc', 'best_epoch-valid_acc_training_loss',
       'best_epoch-valid_acc_ure', 'best_epoch-valid_acc_scel',
       '_wandb_runtime'],
      dtype='object')

In [4]:
import pandas as pd
import numpy as np

for dataset_name in ['uniform-cifar10', 'uniform-cifar20']:
    for algo in ['fwd-u', 'ure-ga-u', 'scl-nl', 'scl-exp', 'l-w', 'l-uw']:
        result = []
        result_es = []
        for seed in [0, 1, 2]:
            # print("seed", seed)
            sub_df = df.loc[(df['algo'] == algo) & (df['seed'] == seed) & (df['dataset_name'] == dataset_name)]
            
            result.append(sub_df.sort_values('valid_acc', ascending=False).iloc[0]['test_acc'])
            result_es.append(sub_df.sort_values('best_epoch-valid_acc_valid_acc', ascending=False).iloc[0]['best_epoch-valid_acc_test_acc'])
            continue
            print(
                dataset_name, seed, algo, 
                # sub_df.sort_values('valid_acc', ascending=False).iloc[0]['valid_acc'],
                sub_df.sort_values('valid_acc', ascending=False).iloc[0]['test_acc'],
                # sub_df.sort_values('valid_acc', ascending=False).iloc[0]['lr'],
                # sub_df.sort_values('test_acc', ascending=False).iloc[0]['test_acc'],
                # sub_df.sort_values('best_epoch-ure_ure').iloc[0]['test_acc']
                sub_df.sort_values('best_epoch-valid_acc_valid_acc', ascending=False).iloc[0]['best_epoch-valid_acc_test_acc'],
                # sub_df.sort_values('best_epoch-valid_acc_valid_acc', ascending=False).iloc[0]['lr']
            )
        print(dataset_name, algo)
        print(round(np.mean(result)*100, 2), round(np.std(result)*100, 2), round(np.mean(result_es)*100, 2), round(np.std(result_es)*100, 2))

uniform-cifar10 fwd-u
69.17 1.22 69.79 1.01
uniform-cifar10 ure-ga-u
54.62 0.6 54.94 1.34
uniform-cifar10 scl-nl
67.15 1.9 68.64 1.98
uniform-cifar10 scl-exp
64.86 0.44 65.4 0.32
uniform-cifar10 l-w
56.21 0.54 59.18 0.46
uniform-cifar10 l-uw
60.88 0.77 62.43 0.46
uniform-cifar20 fwd-u
20.24 0.52 20.62 0.49
uniform-cifar20 ure-ga-u
15.41 0.97 16.59 0.61
uniform-cifar20 scl-nl
20.04 0.48 20.68 0.46
uniform-cifar20 scl-exp
19.4 0.76 21.03 0.64
uniform-cifar20 l-w
14.35 0.74 19.11 1.29
uniform-cifar20 l-uw
16.01 0.89 19.42 0.42
